# Sign Game Server

Game Plan:

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import string

2023-03-08 23:56:29.585048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 23:56:29.730440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 23:56:29.730462: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 23:56:29.759215: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-08 23:56:30.562505: W tensorflow/stream_executor/platform/de

In [2]:
from sign_game.ml.landmarks import Landmarks

ModuleNotFoundError: No module named 'sign_game'

In [36]:
landmark = Landmarks()

NameError: name 'Landmarks' is not defined

In [4]:
# !pip list

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Package                  Version   Editable project location
------------------------ --------- ----------------------------------------------------------
absl-py                  1.4.0
anyio                    3.6.2
appnope                  0.1.3
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
astunparse               1.6.3
attrs                    22.2.0
backcall                 0.2.0
beautifulsoup4           4.11.2
bleach                   6.0.0
cachetools               5.3.0
certifi                  2022.12.7
cffi                     1.15.1
charset-normalizer       3.1.0
click                    8.1.3
comm                     0.1.2
contourpy                1.0.7
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastapi                  0.93.0
fastjsonschema           2.16.3
flatbuffers              23

protobuf 3.11.0

## Dataset Preprocessing

In [43]:
# Dataset
df1 = pd.read_csv('../raw_data/hand-landmarks.csv')

In [44]:
df1.columns

Index(['Unnamed: 0', 'WRIST_x', 'WRIST_y', 'WRIST_z', 'THUMB_CMC_x',
       'THUMB_CMC_y', 'THUMB_CMC_z', 'THUMB_MCP_x', 'THUMB_MCP_y',
       'THUMB_MCP_z', 'THUMB_IP_x', 'THUMB_IP_y', 'THUMB_IP_z', 'THUMB_TIP_x',
       'THUMB_TIP_y', 'THUMB_TIP_z', 'INDEX_FINGER_MCP_x',
       'INDEX_FINGER_MCP_y', 'INDEX_FINGER_MCP_z', 'INDEX_FINGER_PIP_x',
       'INDEX_FINGER_PIP_y', 'INDEX_FINGER_PIP_z', 'INDEX_FINGER_DIP_x',
       'INDEX_FINGER_DIP_y', 'INDEX_FINGER_DIP_z', 'INDEX_FINGER_TIP_x',
       'INDEX_FINGER_TIP_y', 'INDEX_FINGER_TIP_z', 'MIDDLE_FINGER_MCP_x',
       'MIDDLE_FINGER_MCP_y', 'MIDDLE_FINGER_MCP_z', 'MIDDLE_FINGER_PIP_x',
       'MIDDLE_FINGER_PIP_y', 'MIDDLE_FINGER_PIP_z', 'MIDDLE_FINGER_DIP_x',
       'MIDDLE_FINGER_DIP_y', 'MIDDLE_FINGER_DIP_z', 'MIDDLE_FINGER_TIP_x',
       'MIDDLE_FINGER_TIP_y', 'MIDDLE_FINGER_TIP_z', 'RING_FINGER_MCP_x',
       'RING_FINGER_MCP_y', 'RING_FINGER_MCP_z', 'RING_FINGER_PIP_x',
       'RING_FINGER_PIP_y', 'RING_FINGER_PIP_z', 'RING_FINGER

In [45]:
X1 = df1.drop(columns=df1.columns[[0, -1]], axis=1)
X1.columns

Index(['WRIST_x', 'WRIST_y', 'WRIST_z', 'THUMB_CMC_x', 'THUMB_CMC_y',
       'THUMB_CMC_z', 'THUMB_MCP_x', 'THUMB_MCP_y', 'THUMB_MCP_z',
       'THUMB_IP_x', 'THUMB_IP_y', 'THUMB_IP_z', 'THUMB_TIP_x', 'THUMB_TIP_y',
       'THUMB_TIP_z', 'INDEX_FINGER_MCP_x', 'INDEX_FINGER_MCP_y',
       'INDEX_FINGER_MCP_z', 'INDEX_FINGER_PIP_x', 'INDEX_FINGER_PIP_y',
       'INDEX_FINGER_PIP_z', 'INDEX_FINGER_DIP_x', 'INDEX_FINGER_DIP_y',
       'INDEX_FINGER_DIP_z', 'INDEX_FINGER_TIP_x', 'INDEX_FINGER_TIP_y',
       'INDEX_FINGER_TIP_z', 'MIDDLE_FINGER_MCP_x', 'MIDDLE_FINGER_MCP_y',
       'MIDDLE_FINGER_MCP_z', 'MIDDLE_FINGER_PIP_x', 'MIDDLE_FINGER_PIP_y',
       'MIDDLE_FINGER_PIP_z', 'MIDDLE_FINGER_DIP_x', 'MIDDLE_FINGER_DIP_y',
       'MIDDLE_FINGER_DIP_z', 'MIDDLE_FINGER_TIP_x', 'MIDDLE_FINGER_TIP_y',
       'MIDDLE_FINGER_TIP_z', 'RING_FINGER_MCP_x', 'RING_FINGER_MCP_y',
       'RING_FINGER_MCP_z', 'RING_FINGER_PIP_x', 'RING_FINGER_PIP_y',
       'RING_FINGER_PIP_z', 'RING_FINGER_DIP_x', 'RING

In [46]:
df2 = pd.read_csv('../sample_sets/sample_training.csv')

In [47]:
df2.columns

Index(['class_type', 'wristX', 'wristY', 'wristZ', 'thumb_CmcX', 'thumb_CmcY',
       'thumb_CmcZ', 'thumb_McpX', 'thumb_McpY', 'thumb_McpZ', 'thumb_IpX',
       'thumb_IpY', 'thumb_IpZ', 'thumb_TipX', 'thumb_TipY', 'thumb_TipZ',
       'index_McpX', 'index_McpY', 'index_McpZ', 'index_PipX', 'index_PipY',
       'index_PipZ', 'index_DipX', 'index_DipY', 'index_DipZ', 'index_TipX',
       'index_TipY', 'index_TipZ', 'middle_McpX', 'middle_McpY', 'middle_McpZ',
       'middle_PipX', 'middle_PipY', 'middle_PipZ', 'middle_DipX',
       'middle_DipY', 'middle_DipZ', 'middle_TipX', 'middle_TipY',
       'middle_TipZ', 'ring_McpX', 'ring_McpY', 'ring_McpZ', 'ring_PipX',
       'ring_PipY', 'ring_PipZ', 'ring_DipX', 'ring_DipY', 'ring_DipZ',
       'ring_TipX', 'ring_TipY', 'ring_TipZ', 'pinky_McpX', 'pinky_McpY',
       'pinky_McpZ', 'pinky_PipX', 'pinky_PipY', 'pinky_PipZ', 'pinky_DipX',
       'pinky_DipY', 'pinky_DipZ', 'pinky_TipX', 'pinky_TipY', 'pinky_TipZ'],
      dtype='object')

In [48]:
X2 = df2.drop(columns=df2.columns[0], axis=1)
X2.columns=X1.columns
X2.columns

Index(['WRIST_x', 'WRIST_y', 'WRIST_z', 'THUMB_CMC_x', 'THUMB_CMC_y',
       'THUMB_CMC_z', 'THUMB_MCP_x', 'THUMB_MCP_y', 'THUMB_MCP_z',
       'THUMB_IP_x', 'THUMB_IP_y', 'THUMB_IP_z', 'THUMB_TIP_x', 'THUMB_TIP_y',
       'THUMB_TIP_z', 'INDEX_FINGER_MCP_x', 'INDEX_FINGER_MCP_y',
       'INDEX_FINGER_MCP_z', 'INDEX_FINGER_PIP_x', 'INDEX_FINGER_PIP_y',
       'INDEX_FINGER_PIP_z', 'INDEX_FINGER_DIP_x', 'INDEX_FINGER_DIP_y',
       'INDEX_FINGER_DIP_z', 'INDEX_FINGER_TIP_x', 'INDEX_FINGER_TIP_y',
       'INDEX_FINGER_TIP_z', 'MIDDLE_FINGER_MCP_x', 'MIDDLE_FINGER_MCP_y',
       'MIDDLE_FINGER_MCP_z', 'MIDDLE_FINGER_PIP_x', 'MIDDLE_FINGER_PIP_y',
       'MIDDLE_FINGER_PIP_z', 'MIDDLE_FINGER_DIP_x', 'MIDDLE_FINGER_DIP_y',
       'MIDDLE_FINGER_DIP_z', 'MIDDLE_FINGER_TIP_x', 'MIDDLE_FINGER_TIP_y',
       'MIDDLE_FINGER_TIP_z', 'RING_FINGER_MCP_x', 'RING_FINGER_MCP_y',
       'RING_FINGER_MCP_z', 'RING_FINGER_PIP_x', 'RING_FINGER_PIP_y',
       'RING_FINGER_PIP_z', 'RING_FINGER_DIP_x', 'RING

In [49]:
X = X1.append(X2, ignore_index=True)
X.shape

/tmp/ipykernel_14179/2572424810.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X1.append(X2, ignore_index=True)


(2848, 63)

In [64]:
y1=df1['TARGET']
y2=df2['class_type']
y=y1.append(y2, ignore_index=True)
y

/tmp/ipykernel_14179/3843065781.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y=y1.append(y2, ignore_index=True)


0       A
1       A
2       A
3       A
4       A
       ..
2843    J
2844    J
2845    J
2846    J
2847    J
Length: 2848, dtype: object

In [65]:
y.value_counts()

A    112
L    112
W    112
E    112
F    112
V    112
R    112
B    112
M    112
Y    111
U    111
N    111
K    111
I    111
D    111
Z    111
S    110
O    109
H    108
C    108
G    107
T    106
X    105
P    104
Q    104
J    102
dtype: int64

In [66]:
# Encoding letters to numerical, A-Z = 0-25
y = pd.Categorical(y)

In [67]:
y.dtype

CategoricalDtype(categories=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                  'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                  'Y', 'Z'],
, ordered=False)

In [68]:
cat_list = list(y.categories)
cat_list

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [69]:
y = y.codes
display(len(y))
y

2848

array([0, 0, 0, ..., 9, 9, 9], dtype=int8)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [71]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2278, 63), (2278,), (570, 63), (570,))

In [73]:
classes = len(cat_list)
print(classes)

26


In [14]:
# y_train = to_categorical(y_train, num_classes=classes)

In [15]:
# y_test = to_categorical(y_test, num_classes=classes)

In [76]:
X_train = np.reshape(np.array(X_train), (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(np.array(X_test), (X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape, X_test.shape)

(2278, 63, 1) (570, 63, 1)


## CNN Model

### Original CNN

In [ ]:
# Architecture
# model = models.Sequential()
# model.add(layers.Conv1D(filters=512, kernel_size=7, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
# model.add(layers.Conv1D(filters=512, kernel_size=7, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='causal', activation='relu'))
# model.add(layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=256, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.Conv1D(filters=256, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Dropout(0.2))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(26, activation='softmax')) 
#accuracy: 0.89

### Reduced CNN 1

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Dropout(0.2))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(26, activation='softmax'))
# accuracy: 0.71

### Reduced CNN 2

In [ ]:
# model = models.Sequential()
# model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=128, kernel_size=3, strides=1, padding='causal', activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Dropout(0.2))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(26, activation='softmax'))
# accuracy: 0.81

### Reduced CNN 3

In [42]:
model = models.Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=3, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=256, kernel_size=3, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(26, activation='softmax'))
# accuracy: 0.86

### Reduced CNN 4

In [83]:
model = models.Sequential()
model.add(layers.Conv1D(filters=16, kernel_size=3, strides=1, padding='causal', input_shape=X_train.shape[1:3], activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='causal', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(26, activation='softmax'))

In [84]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [85]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 63, 16)            64        
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 31, 16)           0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 31, 32)            1568      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 15, 32)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 15, 64)            6208      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 7, 64)           

In [87]:
es = EarlyStopping(patience=10, restore_best_weights=True)

In [88]:
history = model.fit(X_train, 
                    y_train,
                    epochs=500, 
                    validation_split=0.3, 
                    callbacks=es)

Epoch 1/500
50/50 [==============================] - 1s 12ms/step - loss: 29.5428 - accuracy: 0.0464 - val_loss: 4.7678 - val_accuracy: 0.0643
Epoch 2/500
50/50 [==============================] - 0s 6ms/step - loss: 4.3488 - accuracy: 0.0646 - val_loss: 3.3738 - val_accuracy: 0.0921
Epoch 3/500
50/50 [==============================] - 0s 6ms/step - loss: 3.4946 - accuracy: 0.0784 - val_loss: 3.2493 - val_accuracy: 0.1301
Epoch 4/500
50/50 [==============================] - 0s 6ms/step - loss: 3.3071 - accuracy: 0.1085 - val_loss: 3.1886 - val_accuracy: 0.1213
Epoch 5/500
50/50 [==============================] - 0s 6ms/step - loss: 3.3475 - accuracy: 0.1148 - val_loss: 3.0190 - val_accuracy: 0.1740
Epoch 6/500
50/50 [==============================] - 0s 6ms/step - loss: 2.9749 - accuracy: 0.1593 - val_loss: 2.7864 - val_accuracy: 0.2529
Epoch 7/500
50/50 [==============================] - 0s 6ms/step - loss: 2.7293 - accuracy: 0.2189 - val_loss: 2.4911 - val_accuracy: 0.3202
Epoch 8/500

50/50 [==============================] - 0s 9ms/step - loss: 0.6212 - accuracy: 0.8124 - val_loss: 0.6533 - val_accuracy: 0.8085
Epoch 59/500
50/50 [==============================] - 0s 9ms/step - loss: 0.6124 - accuracy: 0.8024 - val_loss: 0.6733 - val_accuracy: 0.8187
Epoch 60/500
50/50 [==============================] - 0s 9ms/step - loss: 0.6237 - accuracy: 0.8055 - val_loss: 0.6389 - val_accuracy: 0.8216
Epoch 61/500
50/50 [==============================] - 0s 9ms/step - loss: 0.5889 - accuracy: 0.8156 - val_loss: 0.6051 - val_accuracy: 0.8304
Epoch 62/500
50/50 [==============================] - 0s 9ms/step - loss: 0.5451 - accuracy: 0.8181 - val_loss: 0.5860 - val_accuracy: 0.8143
Epoch 63/500
50/50 [==============================] - 0s 9ms/step - loss: 0.5948 - accuracy: 0.8093 - val_loss: 0.5880 - val_accuracy: 0.8260
Epoch 64/500
50/50 [==============================] - 0s 9ms/step - loss: 0.5549 - accuracy: 0.8225 - val_loss: 0.5629 - val_accuracy: 0.8480
Epoch 65/500
50/50 

In [89]:
model.evaluate(X_test, y_test)

18/18 [==============================] - 0s 3ms/step - loss: 0.5533 - accuracy: 0.8754


[0.5533124804496765, 0.8754385709762573]

## Predict

In [90]:
# Dataset
df_pred = pd.read_csv('../sample_sets/sample_validation.csv')

In [91]:
df_pred.columns

Index(['class_type', 'wristX', 'wristY', 'wristZ', 'thumb_CmcX', 'thumb_CmcY',
       'thumb_CmcZ', 'thumb_McpX', 'thumb_McpY', 'thumb_McpZ', 'thumb_IpX',
       'thumb_IpY', 'thumb_IpZ', 'thumb_TipX', 'thumb_TipY', 'thumb_TipZ',
       'index_McpX', 'index_McpY', 'index_McpZ', 'index_PipX', 'index_PipY',
       'index_PipZ', 'index_DipX', 'index_DipY', 'index_DipZ', 'index_TipX',
       'index_TipY', 'index_TipZ', 'middle_McpX', 'middle_McpY', 'middle_McpZ',
       'middle_PipX', 'middle_PipY', 'middle_PipZ', 'middle_DipX',
       'middle_DipY', 'middle_DipZ', 'middle_TipX', 'middle_TipY',
       'middle_TipZ', 'ring_McpX', 'ring_McpY', 'ring_McpZ', 'ring_PipX',
       'ring_PipY', 'ring_PipZ', 'ring_DipX', 'ring_DipY', 'ring_DipZ',
       'ring_TipX', 'ring_TipY', 'ring_TipZ', 'pinky_McpX', 'pinky_McpY',
       'pinky_McpZ', 'pinky_PipX', 'pinky_PipY', 'pinky_PipZ', 'pinky_DipX',
       'pinky_DipY', 'pinky_DipZ', 'pinky_TipX', 'pinky_TipY', 'pinky_TipZ'],
      dtype='object')

In [92]:
X_pred = df_pred.drop(columns = ['class_type'])
X_pred

,wristX,wristY,wristZ,thumb_CmcX,thumb_CmcY,thumb_CmcZ,thumb_McpX,thumb_McpY,thumb_McpZ,thumb_IpX,...,pinky_McpZ,pinky_PipX,pinky_PipY,pinky_PipZ,pinky_DipX,pinky_DipY,pinky_DipZ,pinky_TipX,pinky_TipY,pinky_TipZ
0,67.593429,3675.696075,-0.000018,99.616661,3761.902771,-0.026927,110.972045,3835.375763,-0.026853,121.125584,...,0.072837,114.973211,3854.573547,0.097510,129.272298,3879.857758,0.108352,140.462808,3898.238983,0.113854
1,1436.803349,1737.654764,-0.000152,1578.083852,1765.562633,-0.157858,1641.237340,1777.532579,-0.304467,1646.324825,...,-0.341164,1227.836695,1684.198194,-0.456003,1278.155567,1867.738249,-0.464728,1338.902637,1969.229223,-0.469522
2,648.545802,1607.948065,-0.000153,467.789233,1466.462970,-0.050314,448.115349,1191.233873,-0.097409,611.626387,...,-0.163922,998.544693,1039.246440,-0.247517,893.199384,1167.122245,-0.246745,882.732987,1264.709234,-0.227733
3,833.924472,1559.609413,-0.000164,648.933053,1397.195578,-0.041743,647.326112,1105.966210,-0.078414,848.145723,...,-0.155674,1185.740709,996.946692,-0.238493,1068.991184,1137.645721,-0.229178,1069.342017,1233.988047,-0.199588
4,2585.928783,2014.379934,-0.000058,2489.925442,1913.919084,-0.022915,2510.127912,1722.629985,-0.031609,2600.052752,...,-0.016604,2819.434075,1749.852462,-0.053624,2737.283750,1812.189479,-0.053119,2664.588332,1866.325679,-0.041351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1354.846124,1946.291035,-0.000071,1436.005777,1831.889364,0.112920,1450.998651,1713.268372,0.166175,1466.387431,...,0.008276,1312.858226,1421.120537,0.016607,1307.488609,1353.245492,0.018358,1307.269927,1301.262153,0.013307
200,900.431514,1510.550857,-0.000056,1124.763131,1308.396101,0.072219,1196.975231,1071.930170,0.099373,1109.926462,...,-0.109490,413.159043,927.854598,-0.097462,296.604931,811.812997,-0.115920,186.776280,710.417211,-0.161983
201,72.850646,3754.295013,-0.000019,135.652102,3874.291260,-0.027033,141.803033,3974.507446,-0.027239,144.245676,...,0.094596,69.929686,3992.984802,0.126661,85.686087,4023.706146,0.142894,99.380424,4046.099579,0.154124
202,1260.964149,2159.256546,0.000274,1335.243220,1851.975826,0.219779,1252.857707,1601.350079,0.317800,1106.594398,...,-0.040661,391.540262,1692.171650,0.013872,211.291803,1652.817776,0.028499,59.546666,1606.800190,0.009594


In [93]:
X_pred = np.reshape(np.array(X_pred), (X_pred.shape[0], X_pred.shape[1], 1))
display(X_pred.shape)
display(type(X_pred))

(204, 63, 1)

numpy.ndarray

In [94]:
y_act=df_pred['class_type']
y_act

0      N
1      N
2      N
3      N
4      N
      ..
199    J
200    J
201    J
202    J
203    J
Name: class_type, Length: 204, dtype: object

In [95]:
y_pred=model.predict(X_pred)
y_pred

7/7 [==============================] - 0s 3ms/step


array([[4.06886311e-03, 8.07576329e-02, 3.43941785e-02, ...,
        1.48171559e-04, 3.07850532e-05, 5.21077891e-04],
       [1.92235783e-02, 1.71487369e-02, 1.03487605e-02, ...,
        2.26096913e-01, 3.82540450e-02, 2.07605828e-02],
       [4.09215827e-05, 1.77528337e-03, 2.14708946e-03, ...,
        3.07371764e-08, 2.15204828e-03, 4.41824377e-05],
       ...,
       [3.60278529e-03, 8.50392878e-02, 4.12154943e-02, ...,
        1.64694065e-04, 2.31153099e-05, 4.81841620e-04],
       [6.05830969e-03, 7.66794642e-07, 2.46762938e-05, ...,
        4.46307195e-06, 4.31345627e-02, 1.02261811e-05],
       [1.32140284e-03, 2.06737113e-07, 1.29862919e-05, ...,
        9.54463417e-07, 2.39913329e-03, 1.56078897e-06]], dtype=float32)

In [96]:
preds = [cat_list[np.argmax(pred)] for pred in y_pred]
preds

['T',
 'W',
 'E',
 'N',
 'N',
 'Q',
 'N',
 'J',
 'R',
 'R',
 'R',
 'R',
 'L',
 'T',
 'R',
 'U',
 'W',
 'B',
 'B',
 'F',
 'B',
 'B',
 'K',
 'D',
 'B',
 'M',
 'J',
 'I',
 'Y',
 'S',
 'I',
 'I',
 'F',
 'I',
 'F',
 'F',
 'F',
 'F',
 'B',
 'F',
 'H',
 'L',
 'H',
 'H',
 'H',
 'O',
 'H',
 'H',
 'E',
 'J',
 'E',
 'E',
 'E',
 'E',
 'C',
 'E',
 'B',
 'U',
 'U',
 'U',
 'U',
 'R',
 'V',
 'U',
 'E',
 'J',
 'E',
 'M',
 'M',
 'K',
 'M',
 'Z',
 'N',
 'K',
 'V',
 'X',
 'J',
 'X',
 'X',
 'L',
 'T',
 'K',
 'N',
 'K',
 'K',
 'V',
 'K',
 'K',
 'K',
 'K',
 'Z',
 'N',
 'N',
 'Q',
 'Q',
 'Q',
 'Q',
 'Y',
 'Y',
 'Y',
 'Y',
 'M',
 'Y',
 'Y',
 'I',
 'S',
 'S',
 'M',
 'S',
 'S',
 'N',
 'N',
 'R',
 'A',
 'G',
 'G',
 'G',
 'T',
 'G',
 'G',
 'N',
 'A',
 'A',
 'A',
 'Y',
 'D',
 'A',
 'A',
 'L',
 'O',
 'O',
 'O',
 'O',
 'O',
 'T',
 'O',
 'O',
 'T',
 'N',
 'Y',
 'T',
 'J',
 'A',
 'N',
 'T',
 'V',
 'V',
 'V',
 'V',
 'V',
 'V',
 'N',
 'V',
 'Z',
 'Z',
 'G',
 'G',
 'Z',
 'Z',
 'Z',
 'O',
 'O',
 'O',
 'C',
 'C',
 'O',
 'D'

In [98]:
correct=0
comparisons=list(zip(preds, y_act))
for comparison in comparisons:
    if comparison[0]==comparison[1]:
        correct+=1
correct_percent=correct/len(y_act)
print('Correct Predictions: ', correct_percent)

Correct Predictions:  0.6274509803921569


In [97]:
np.unique(np.array(preds))

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
      dtype='<U1')